<a href="https://colab.research.google.com/github/thunchanokbow/Inventory-amazon/blob/main/Inventory2Q2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xzvf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark==1.3.0

In [ ]:
# Install PySpark in python
!pip install pyspark==3.1.2

In [3]:
# Set enviroment variable
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
import pandas as pd
# Create a Spark session.
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
# from pyspark.sql.functions import col
import pyspark.sql.functions as f
import pyspark.sql.types as T
# PostgreSQL connection
from sqlalchemy import create_engine
from sqlalchemy import inspect
import psycopg2
import getpass

# Pulling Data from PostgreaSQL database

In [ ]:
# Connection to Database PostgreSQL
host = "azure-amazon.postgres.database.azure.com"
port = "5432"
dbname = "db-amazon"
user = input("user name: ")
pwd = getpass.getpass("password: ")
connection_string = f'postgresql+psycopg2://{user}:{pwd}@{host}:{port}/{dbname}'

In [ ]:
# Get table from Database
engine = create_engine(connection_string)
insp = inspect(engine)
insp.get_table_names()

In [ ]:
# Query product_price table
product_price = spark.sql("SELECT * FROM product_price")
# Query sales_report table
sales_report = spark.sql("SELECT * FROM sales_report")

##Convert the `Amount` column to Integer datatype

In [ ]:
product_price.printSchema()

root
 |-- index: integer (nullable = true)
 |-- sku: string (nullable = true)
 |-- Design No.: string (nullable = true)
 |-- Stock: double (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Amount: double (nullable = true)



In [ ]:
# Convert the Amount column to integer.
product_price = product_price.withColumn("Amount", product_price["Amount"].cast("Integer"))
product_price.printSchema()
product_price.count()

root
 |-- index: integer (nullable = true)
 |-- sku: string (nullable = true)
 |-- Design No.: string (nullable = true)
 |-- Stock: double (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Amount: integer (nullable = true)



9406

In [ ]:
sales_report.printSchema()
sales_report.count()

root
 |-- index: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Sales Channel : string (nullable = true)
 |-- Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Qty: integer (nullable = true)



128975

#Analyze Best and worst selling SKU items

In [7]:
# Group the data by SKU and calculate the sum of Qty.
best_items = sales_report.groupBy("SKU").sum("Qty")

In [8]:
# Sort the DataFrame by Qty column in descending order.
best_items = best_items.sort("sum(Qty)", ascending=False)
best_items.show()

+---------------+--------+
|            SKU|sum(Qty)|
+---------------+--------+
|   JNE3797-KR-L|     661|
|   JNE3797-KR-M|     561|
|   JNE3797-KR-S|     503|
|   JNE3405-KR-L|     485|
|    J0230-SKD-M|     468|
|    J0230-SKD-S|     421|
|  JNE3797-KR-XL|     415|
|   JNE3405-KR-S|     399|
|  JNE3797-KR-XS|     386|
|SET268-KR-NP-XL|     373|
| JNE3797-KR-XXL|     359|
|JNE3797-KR-XXXL|     353|
| SET268-KR-NP-S|     350|
| SET268-KR-NP-L|     348|
|   JNE3405-KR-M|     344|
|   JNE3567-KR-M|     291|
| SET183-KR-DH-M|     289|
|    J0003-SET-S|     281|
|    J0230-SKD-L|     272|
| SET324-KR-NP-M|     268|
+---------------+--------+
only showing top 20 rows



In [ ]:
# Sort the DataFrame by Qty column in ascending order.
worst_items = best_items.sort("sum(Qty)", descending=False)
worst_items.show()

+-----------------+--------+
|              SKU|sum(Qty)|
+-----------------+--------+
|   SET235-KR-PP-S|       0|
|      J0086-DR-XL|       0|
|      J0162-SKD-M|       0|
|    JNE3557-KR-XL|       0|
|      BTM046-PP-M|       0|
|      J0053-KR-XS|       0|
|       J0292-TP-L|       0|
|     BTM002-B-XXL|       0|
|         BL091-XL|       0|
| SET043-KR-NP-XXL|       0|
|   J0137-SET-XXXL|       0|
|  NW002-TP-PJ-XXL|       0|
|  AN204-PURPLE-XL|       0|
|    JNE3602-KR-XS|       0|
|   SET055-KR-NP-S|       0|
| NW032-ST-CP-XXXL|       0|
|    JNE3284-KR-XL|       0|
|      J0155-KR-XS|       0|
|PSET058-KR-NP-6XL|       0|
|     JNE3359-KR-L|       0|
+-----------------+--------+
only showing top 20 rows



In [ ]:
sales_report.describe().show()
# The dataset covers months 3 to 6

+-------+------------------+-------------------+--------+---------+--------------+------+--------------+--------+------+-------------------+
|summary|             index|           Order ID|    Date|   Status|Sales Channel | Style|           SKU|Category|  Size|                Qty|
+-------+------------------+-------------------+--------+---------+--------------+------+--------------+--------+------+-------------------+
|  count|            128975|             128975|  128975|   128975|        128975|128975|        128975|  128975|128975|             128975|
|   mean|           64487.0|               null|    null|     null|          null|  null|          null|    null|  null| 0.9044310912967629|
| stddev|37232.019821653514|               null|    null|     null|          null|  null|          null|    null|  null|0.31335358565014465|
|    min|                 0|171-0000547-8192359|03-31-22|Cancelled|     Amazon.in| AN201|   AN201-RED-M|  Blouse|   3XL|                  0|
|    max|    

#Analyze Peak periods with highest sales of the year

##Convert the `Date` column to timestamp datatype format `MM-dd-yyyy`

In [ ]:
# Covert to Pandas Dataframe.
high_report = sales_report
high_report_pd = high_report.toPandas()

In [ ]:
# Convert the Date column to a datetime dtype New_date column.
high_report_pd["New_date"] = pd.to_datetime(high_report_pd["Date"])

In [ ]:
# Convert the Pandas Data Frame to Spark Data Frame.
high_report = spark.createDataFrame(high_report_pd)
high_report.printSchema

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


<bound method DataFrame.printSchema of DataFrame[index: bigint, Order ID: string, Date: string, Status: string, Sales Channel : string, Style: string, SKU: string, Category: string, Size: string, Qty: bigint, New_date: timestamp]>

In [ ]:
# Create a new column called order_month use the month function to extract the month of the date column.
high_report = high_report.withColumn("order_month",
                                     f.month(high_report["New_date"]))
high_report.show()

+-----+-------------------+--------+--------------------+--------------+--------+-------------------+-------------+----+---+-------------------+-----------+
|index|           Order ID|    Date|              Status|Sales Channel |   Style|                SKU|     Category|Size|Qty|           New_date|order_month|
+-----+-------------------+--------+--------------------+--------------+--------+-------------------+-------------+----+---+-------------------+-----------+
|    0|405-8078784-5731545|04-30-22|           Cancelled|     Amazon.in|  SET389|     SET389-KR-NP-S|          Set|   S|  0|2022-04-30 00:00:00|          4|
|    1|171-9198151-1101146|04-30-22|Shipped - Deliver...|     Amazon.in| JNE3781|    JNE3781-KR-XXXL|        kurta| 3XL|  1|2022-04-30 00:00:00|          4|
|    2|404-0687676-7273146|04-30-22|             Shipped|     Amazon.in| JNE3371|      JNE3371-KR-XL|        kurta|  XL|  1|2022-04-30 00:00:00|          4|
|    3|403-9615377-8133951|04-30-22|           Cancelled| 

##Best products revenue

In [ ]:
# Group the data by SKU and calculate the sum of Qty.
best_items = high_report.groupBy("SKU").sum("Qty")

In [ ]:
# Join Inner product_price to best_revenue dataframe
best_revenue = best_items.join(product_price,
                               on=["SKU"], how="inner")
best_revenue.show()
best_revenue.count()

+----------------+--------+-----+----------+-----+----------+----+-----------+------+
|             SKU|sum(Qty)|index|Design No.|Stock|  Category|Size|      Color|Amount|
+----------------+--------+-----+----------+-----+----------+----+-----------+------+
|   MEN5021-KR-XL|      26| 6877|   MEN5021|  6.0|     KURTA|  XL|     Orange|   764|
|JNE1998-KR-311-S|       4| 3344|   JNE1998| 54.0|     KURTA|   S| Multicolor|   295|
| SET293-KR-NP-XL|      53| 8422|    SET293|  4.0| KURTA SET|  XL|       Blue|   702|
|  SET374-KR-NP-M|      85| 8900|    SET374| 41.0|       SET|   M|    Mustard|   666|
|  SET145-KR-NP-L|      28| 7671|    SET145|  4.0| KURTA SET|   L|       Blue|   774|
|  SET339-KR-NP-S|      19| 8698|    SET339|  8.0|       SET|   S|     Maroon|   736|
|      J0094-KR-L|       6| 1089|     J0094|  4.0|     KURTA|   L|       Blue|   568|
|          SAR066|       2| 7294|    SAR066|  0.0|     SAREE|FREE|       Teal|   667|
|   JNE3790-KR-XL|       8| 6031|   JNE3790| 12.0|    

6214

In [ ]:
# Save best_revenue format csv to bucket
best_revenue.coalesce(1).write.csv('best_revenue', header = True)

## Peak sales time of this quarter

In [ ]:
# Group the data by order_month and calculate the sum of quantity.
monthly_sales = high_report.groupBy("order_month").sum("Qty")
monthly_sales = monthly_sales.sort("order_month", ascending=True)
monthly_sales.show()

+-----------+--------+
|order_month|sum(Qty)|
+-----------+--------+
|          3|     156|
|          4|   44206|
|          5|   37926|
|          6|     927|
+-----------+--------+



In [ ]:
# Save monthly_sales format csv to bucket
monthly_sales.coalesce(1).write.csv('monthly_sales', header = True)

#Analyze Excess Inventory Value

In [ ]:
# Filter the sum(Qty) column is equal to 0 and display the SKU column.
worst_list = worst_items.where(worst_items["sum(Qty)"] == 0).select("SKU")
worst_list.show()
worst_list.count()

+-----------------+
|              SKU|
+-----------------+
| NW032-ST-CP-XXXL|
|    JNE3284-KR-XL|
|     BTM002-B-XXL|
|      J0155-KR-XS|
|PSET058-KR-NP-6XL|
|         BL091-XL|
|    JNE3557-KR-XL|
|     JNE3359-KR-L|
| SET043-KR-NP-XXL|
|   J0137-SET-XXXL|
|      BTM046-PP-M|
|      J0086-DR-XL|
|  NW002-TP-PJ-XXL|
|  AN204-PURPLE-XL|
|      J0053-KR-XS|
|    JNE3602-KR-XS|
|   SET055-KR-NP-S|
|       J0292-TP-L|
|      J0162-SKD-M|
|   SET235-KR-PP-S|
+-----------------+
only showing top 20 rows



59

In [ ]:
worst_list.take(54)

[Row(SKU='SET247-KR-SHA-XXXL'),
 Row(SKU='J0374-KR-M'),
 Row(SKU='JNE3284-KR-XL'),
 Row(SKU='JNE3403-KR-XXL'),
 Row(SKU='J0109-SKD-XS'),
 Row(SKU='JNE2145-KR-A-L'),
 Row(SKU='BTM002-B-XXL'),
 Row(SKU='J0155-KR-XS'),
 Row(SKU='PSET058-KR-NP-6XL'),
 Row(SKU='BL091-XL'),
 Row(SKU='JNE3557-KR-XL'),
 Row(SKU='SET043-KR-NP-XXL'),
 Row(SKU='J0086-DR-XL'),
 Row(SKU='SAR060'),
 Row(SKU='JNE3423-KR-A-S'),
 Row(SKU='AN204-PURPLE-XL'),
 Row(SKU='JNE3897-KR-M'),
 Row(SKU='J0292-TP-L'),
 Row(SKU='SET235-KR-PP-S'),
 Row(SKU='JNE3447-KR-XL'),
 Row(SKU='AN210-NAVY-M'),
 Row(SKU='BL104-XL'),
 Row(SKU='JNE2294-KR-A-XS'),
 Row(SKU='SAR182'),
 Row(SKU='BL100-XXL'),
 Row(SKU='J0080-TP-L'),
 Row(SKU='J0073-KR-S'),
 Row(SKU='J0184-KR-A-L'),
 Row(SKU='JNE3891-TP-XXL'),
 Row(SKU='J0302-DR-XL'),
 Row(SKU='SET199-KR-NP-XS'),
 Row(SKU='J0057-DR-XXXL'),
 Row(SKU='J0122-TP-L'),
 Row(SKU='J0173-DR-XL'),
 Row(SKU='JNE3364-KR-1051-XS'),
 Row(SKU='JNE3483-KR-M'),
 Row(SKU='SAR166'),
 Row(SKU='SET083-KR-PP-L'),
 Row(SKU=

In [ ]:
# Matching rows in product_price that have a same value of the SKU column
worst_list_price = worst_list.join(product_price,
                                   on=["SKU"], how="inner")
worst_list_price.show()

+-----------------+-----+----------+-----+-------------+----+--------------+------+
|              SKU|index|Design No.|Stock|     Category|Size|         Color|Amount|
+-----------------+-----+----------+-----+-------------+----+--------------+------+
| NW032-ST-CP-XXXL| 7113|     NW032|  7.0|   NIGHT WEAR|XXXL|         White|   992|
|    JNE3284-KR-XL| 3647|   JNE3284|  4.0|        KURTA|  XL|         Peach|   308|
|      J0155-KR-XS| 1500|     J0155| 62.0|        KURTA|  XS|         Green|   292|
|PSET058-KR-NP-6XL| 7243|   PSET058|  0.0|          SET| 6XL|          Teal|  1216|
|         BL091-XL|  164|     BL091|  3.0|       BLOUSE|  XL|    Multicolor|   292|
|    JNE3557-KR-XL| 4735|   JNE3557| 36.0|        KURTA|  XL|         Black|   292|
|     JNE3359-KR-L| 3746|   JNE3359|  7.0|        KURTA|   L|          Grey|   390|
| SET043-KR-NP-XXL| 7446|    SET043| 60.0|    KURTA SET| XXL|Turquoise Blue|   292|
|   J0137-SET-XXXL| 1377|     J0137| 12.0|    KURTA SET|XXXL|         White|

##Convert the `Stock` column to Integer datatype

In [ ]:
worst_list_price.printSchema()

root
 |-- SKU: string (nullable = true)
 |-- index: integer (nullable = true)
 |-- Design No.: string (nullable = true)
 |-- Stock: double (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Amount: integer (nullable = true)



In [ ]:
# Convert the Stock column to integer.
worst_list_price = worst_list_price.withColumn("Stock", worst_list_price["Stock"].cast("Integer"))
worst_list_price.printSchema()

root
 |-- SKU: string (nullable = true)
 |-- index: integer (nullable = true)
 |-- Design No.: string (nullable = true)
 |-- Stock: integer (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Amount: integer (nullable = true)



In [ ]:
# Calculate the total value of the product and put it in the total column.
worst_list_price = worst_list_price.withColumn("Inventory_value", worst_list_price["Stock"] * worst_list_price["Amount"])
worst_list_price.show()

+-----------------+-----+----------+-----+-------------+----+--------------+------+---------------+
|              SKU|index|Design No.|Stock|     Category|Size|         Color|Amount|Inventory_value|
+-----------------+-----+----------+-----+-------------+----+--------------+------+---------------+
| NW032-ST-CP-XXXL| 7113|     NW032|    7|   NIGHT WEAR|XXXL|         White|   992|           6944|
|    JNE3284-KR-XL| 3647|   JNE3284|    4|        KURTA|  XL|         Peach|   308|           1232|
|      J0155-KR-XS| 1500|     J0155|   62|        KURTA|  XS|         Green|   292|          18104|
|PSET058-KR-NP-6XL| 7243|   PSET058|    0|          SET| 6XL|          Teal|  1216|              0|
|         BL091-XL|  164|     BL091|    3|       BLOUSE|  XL|    Multicolor|   292|            876|
|    JNE3557-KR-XL| 4735|   JNE3557|   36|        KURTA|  XL|         Black|   292|          10512|
|     JNE3359-KR-L| 3746|   JNE3359|    7|        KURTA|   L|          Grey|   390|           2730|


In [ ]:
# Save Inventory_Value format csv to bucket
worst_list_price.coalesce(1).write.csv('Inventory_Value', header = True)